In [1]:
import chromadb

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

# Open document

In [11]:
loader = TextLoader("data/Lincoln_State_of_Union_1862.txt")

In [17]:
lincoln_speech_doc = loader.load()[0]

# Split document

In [16]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)

In [18]:
chunked_docs = splitter.split_documents([lincoln_speech_doc])

Created a chunk of size 608, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500


# Create and store embeddings

In [20]:
embedding_function = OpenAIEmbeddings()

In [22]:
db = Chroma.from_documents(chunked_docs, embedding_function, persist_directory=".speech_db")

In [23]:
db.persist()

# query database

In [25]:
db_connection = Chroma(persist_directory=".speech_db", embedding_function=embedding_function)

In [ ]:
db_connection.sim